# Setup

In [1]:
using RigidBodyDynamics
using ForwardDiff
using Compat.Test
srand(1);

# Jacobians with respect to $q$ and $v$ - the simple way

First, we'll load our trusty double pendulum from a URDF:

In [2]:
const mechanism = parse_urdf(Float64, "../test/urdf/Acrobot.urdf")
remove_fixed_tree_joints!(mechanism)
mechanism

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

Of course, we can create a `MechanismState` for the double pendulum, and compute its momentum in some random state:

In [3]:
float64state = MechanismState(mechanism)
rand!(float64state)
const q = configuration(float64state) # for future use
const v = velocity(float64state) # for future use
momentum(float64state)

Momentum expressed in "world":
angular: [0.0801508, 1.43084, 0.159513], linear: [-0.697851, 0.0, 0.338924]

But now suppose we want the Jacobian of momentum with respect to the joint velocity vector $v$. We can do this using the `ForwardDiff.Dual` type and the `ForwardDiff.jacobian` function. The ForwardDiff package implements forward-mode [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation).

To use `ForwardDiff.jacobian` we need to create a function that maps `v` (as a `Vector`) to momentum (as a `Vector`):

In [4]:
function mom(v::AbstractVector{T}) where T
    # create a `MechanismState` that can handle the element type of `v` (which will be some `ForwardDiff.Dual`):
    state = MechanismState{T}(mechanism)
    
    # set the state variables:
    set_configuration!(state, q)
    set_velocity!(state, v)
    
    # return momentum converted to an `Array` (as this is the format that ForwardDiff expects)
    Array(momentum(state))
end

mom (generic function with 1 method)

Let's first check that the function returns the same thing we got from `float64state`:

In [5]:
@test mom(v) == Array(momentum(float64state))

Test Passed

That works, so now let's compute the Jacobian with `ForwardDiff`:

In [6]:
J = ForwardDiff.jacobian(mom, v)

6×2 Array{Float64,2}:
  0.252338   0.157137
  4.51855    2.25777 
  0.505187   0.194443
 -2.21197   -0.777771
  0.0        0.0     
  1.06794    0.628547

At this point we note that the matrix `J` is simply the momentum matrix (in world frame) of the `Mechanism`. In this case, RigidBodyDynamics.jl has a specialized algorithm for computing this matrix, so let's verify the results:

In [7]:
A = momentum_matrix(float64state)

MomentumMatrix expressed in "world":
[0.252338 0.157137; 4.51855 2.25777; 0.505187 0.194443; -2.21197 -0.777771; 0.0 0.0; 1.06794 0.628547]

In [8]:
@test J ≈ Array(A) atol = 1e-12

Test Passed

Gradients with respect to $q$ can be computed in similar fashion.

# Improving performance

Ignoring the fact that we have a specialized method available, let's look at the performance of using `ForwardDiff.jacobian`.

In [9]:
using BenchmarkTools
@benchmark ForwardDiff.jacobian($mom, $v)

BenchmarkTools.Trial: 
  memory estimate:  46.75 KiB
  allocs estimate:  703
  --------------
  minimum time:     106.915 μs (0.00% GC)
  median time:      123.764 μs (0.00% GC)
  mean time:        130.577 μs (6.15% GC)
  maximum time:     4.417 ms (94.65% GC)
  --------------
  samples:          10000
  evals/sample:     1

That's not great. Note all the allocations. We can do better by making the following modifications:

1. use an in-place version of the `jacobian` function, `ForwardDiff.jacobian!`
2. reimplement our `mom` function to be in-place as well
3. don't create a new `MechanismState` every time

The third point is especially important; creating a `MechanismState` is expensive!

To facillitate reuse of `MechanismState`s while keeping the code nice and generic, we can use a `StateCache` object.
`StateCache` is a container that stores `MechanismState`s of various types (associated with one `Mechanism`), and will ease the process of using `ForwardDiff`.
Creating one is easy:

In [10]:
const statecache = StateCache(mechanism)

StateCache{…}(…)

`MechanismState`s of a given type can be accessed as follows (note that if a `MechanismState` of a certain type is already available, it will be reused):

In [11]:
float32state = statecache[Float32]

MechanismState{Float32, Float64, Float64, …}(…)

In [12]:
@test float32state === statecache[Float32]

Test Passed

Now we'll use the `StateCache` to reimplement `mom`, making it in-place as well:

In [13]:
function mom!(out::AbstractVector, v::AbstractVector{T}) where T
    # retrieve a `MechanismState` that can handle the element type of `v`:
    state = statecache[T]
    
    # set the state variables:
    set_configuration!(state, q)
    set_velocity!(state, v)
    
    # compute momentum and store it in `out`
    m = momentum(state)
    copy!(out, [angular(m); linear(m)])
end

mom! (generic function with 1 method)

Check that the in-place version works as expected on `Float64` inputs:

In [14]:
const out = zeros(6) # where we'll be storing our results
mom!(out, v)
@test out == Array(momentum(float64state))

Test Passed

And use `ForwardDiff.jacobian!` to compute the Jacobian:

In [15]:
const result = DiffResults.JacobianResult(out, v)
const config = ForwardDiff.JacobianConfig(mom!, out, v)
ForwardDiff.jacobian!(result, mom!, out, v, config)
J = DiffResults.jacobian(result)
@test J ≈ Array(A) atol = 1e-12

Test Passed

Let's check the performance again:

In [16]:
@benchmark ForwardDiff.jacobian!($result, $mom!, $out, $v, $config)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.026 μs (0.00% GC)
  median time:      2.043 μs (0.00% GC)
  mean time:        2.078 μs (0.00% GC)
  maximum time:     4.953 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     9

That's much better. Do note that the specialized algorithm is still faster:

In [17]:
A2 = momentum_matrix(float64state)
@benchmark begin
    set_configuration!($float64state, $q)
    set_velocity!($float64state, $v)
    momentum_matrix!($A2, $float64state)
end

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     696.804 ns (0.00% GC)
  median time:      699.351 ns (0.00% GC)
  mean time:        709.050 ns (0.00% GC)
  maximum time:     1.135 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     148

# Time derivatives

We can also use ForwardDiff to compute time derivatives. Let's verify that energy is conserved for the double pendulum in the absence of nonconservative forces (like damping). That is, we expect that the time derivative of the pendulum's total energy is zero when its state evolves according to the passive dynamics.

Let's first compute the joint acceleration vector $\dot{v}$ using the passive dynamics:

In [18]:
dynamicsresult = DynamicsResult(mechanism)
set_configuration!(float64state, q)
set_velocity!(float64state, v)
dynamics!(dynamicsresult, float64state)
v̇ = dynamicsresult.v̇

2-element RigidBodyDynamics.CustomCollections.SegmentedVector{RigidBodyDynamics.JointID,Float64,Base.OneTo{RigidBodyDynamics.JointID},Array{Float64,1}}:
  0.0796808
 -4.79883  

Now for the time derivative of total energy. ForwardDiff has a `derivative` function that can be used to take derivatives of functions that map a scalar to a scalar. But in this example, we'll instead use ForwardDiff's `Dual` type directly. `ForwardDiff.Dual` represents a (potentially multidimensional) dual number, i.e., a type that stores both the value of a function evaluated at a certain point, as well as the partial derivatives of the function, again evaluated at the same point. See the [ForwardDiff documentation](http://www.juliadiff.org/ForwardDiff.jl/stable/dev/how_it_works.html) for more information.

We'll create a vector of `Dual`s representing the value and derivative of $q(t)$:

In [19]:
q̇ = v
q_dual = ForwardDiff.Dual.(q, q̇)

2-element Array{ForwardDiff.Dual{Void,Float64,1},1}:
 Dual{Void}(0.297288,0.312707)  
 Dual{Void}(0.382396,0.00790928)

**Note**: for the double pendulum, $\dot{q} = v$, but this is not the case in general for `Mechanism`s created using RigidBodyDynamics.jl. For example, the `QuaternionSpherical` joint type uses a unit quaternion to represent the joint configuration, but angular velocity (in body frame) to represent velocity. In general $\dot{q}$ can be computed from the velocity vector $v$ stored in a `MechanismState` using

```julia
configuration_derivative(::MechanismState)
```

or its in-place variant, `configuration_derivative!`.

We'll do the same thing for $v(t)$:

In [20]:
v_dual = ForwardDiff.Dual.(v, v̇)

2-element Array{ForwardDiff.Dual{Void,Float64,1},1}:
 Dual{Void}(0.312707,0.0796808) 
 Dual{Void}(0.00790928,-4.79883)

Now we're ready to compute the total energy (kinetic + potential) using these `ForwardDiff.Dual` inputs. We'll use our `StateCache` again:

In [21]:
T = eltype(q_dual)
state = statecache[T]
set_configuration!(state, q_dual)
set_velocity!(state, v_dual)
energy_dual = kinetic_energy(state) + gravitational_potential_energy(state)

Dual{Void}(-21.472905435008563,4.440892098500626e-16)

Note that the result type of `energy_dual` is again a `ForwardDiff.Dual`. We can extract the energy and its time derivative (mechanical power) from `energy_dual` as follows:

In [22]:
energy = ForwardDiff.value(energy_dual)
partials = ForwardDiff.partials(energy_dual)
power = partials[1];

So the total energy in the system is:

In [23]:
energy

-21.472905435008563

**Note**: the total energy is negative because the origin of the world frame is used as a reference for computing gravitational potential energy, i.e., the center of mass of the double pendulum is somewhere below this origin.

And we can verify that, indeed, there is no power flow into or out of the system:

In [24]:
@test power ≈ 0 atol = 1e-14

Test Passed